In [ ]:
import json
import pandas as pd
import os
#!pip install openai -U
from openai import OpenAI
from enum import Enum
from typing import List
from pydantic import BaseModel, Field
client = OpenAI(api_key=os.environ.get('OPENAI_API_KEY'))
model = "gpt-4o-2024-08-06"

In [82]:
# Get human emotions

# Define the Pydantic model for the API response
class EmotionsResponse(BaseModel):
    Characteristics: List[str] = Field(None, description="List of non-redundant human emotions.")

def get_emotions(model: str) -> List[str]:
    """Gets a list of 50 non-redundant human emotions using the specified gpt model."""
    
    # Define system and user prompts
    system_prompt = "Find 50 non-redundant and different human emotions. "\
    "For example, from each of these four emotion groups, only select one representative"\
    ": 1: joy, happiness, 2: Shame, Embarrassment, "\
    "3: Envy, Jealousy , 4: Hate, disgust, hatered, Resentment. "\
    "So on and so forth."

    user_prompt = "Select 50 non-redundant and different human emotions."

    try:
        #Call the API to get the completion
        completion = client.beta.chat.completions.parse(
            model= model,
            messages=[
                {"role": "system", "content": "Be a helpful assistant."},
                {"role": "system", "content": system_prompt},
                {"role": "system", "content": "Did you accidentally select multiple emotions from the four emotion groups? If so, keep only one representative from each group and drop the rest."},
                {"role": "system", "content": "Check again for redundancy. Remember, you must only select emotions describing non-redundant human feelings."},
                {"role": "user", "content": user_prompt}
            ],
            response_format=EmotionsResponse
        )

        #output returns in the defined pydantic style
        output = completion.choices[0].message.parsed
        return output.json()
    
    except Exception as e:
        # Handle exceptions such as API errors, etc
        print(f"An error occurred: {e}")
        return []

# Example usage
emotions = get_emotions(model= model)
emotions_ls = list(json.loads(emotions).values())[0]


'{"Characteristics": ["Joy", "Shame", "Envy", "Disgust", "Love", "Fear", "Surprise", "Anger", "Sadness", "Hope", "Guilt", "Gratitude", "Contempt", "Pride", "Sympathy", "Awe", "Relief", "Frustration", "Confusion", "Regret", "Amusement", "Empathy", "Despair", "Nostalgia", "Anticipation", "Loneliness", "Courage", "Trust", "Betrayal", "Disappointment", "Curiosity", "Contentment", "Admiration", "Euphoria", "Greed", "Humility", "Resilience", "Skepticism", "Insecurity", "Melancholy", "Optimism", "Cynicism", "Doubt", "Resentment", "Pleasure", "Worry", "Compassion", "Vulnerability", "Serenity"]}'

In [83]:
# #to check emotion redundancy by looking at example groups 
#[i for i in list(json.loads(emotions).values())[0] if i in ['Joy', 'Happiness', 'Shame', 'Embarrassment', 'Envy', 'Jealousy' , 'Hate', 'disgust', 'hatered', 'Resentment']]

['Joy', 'Shame', 'Envy', 'Resentment']

In [100]:
#Get 100 best selling American clothing brands 

# Define the Pydantic model for the API response
class BrandsResponse(BaseModel):
    Brands: List[str] = Field(None, description="Brands as a list of strings.")

def get_brands(model: str) -> List[str]:
    """Get 100 best selling American clothing brands using the specified gpt model."""

    try:
        #Call the API to get the completion
        completion = client.beta.chat.completions.parse(
            model= model,
            messages=[
                {"role": "system", "content": "Be a helpful assistant."},
                {"role": "system", "content": "Find 100 non-redundant best selling American clothing brands."},
                {"role": "system", "content": "DONT MAKE ANY MISTAKES, check if you did any."},
                {"role": "user", "content": "Give me 100 best selling American clothing brands."}
            ],
            response_format=BrandsResponse
        )

        #output returns in the defined pydantic style
        output = completion.choices[0].message.parsed
        return output.json()
    
    except Exception as e:
        # Handle exceptions such as API errors, etc
        print(f"An error occurred: {e}")
        return []

# Example usage
brands = get_brands(model= model)
brands_ls = list(json.loads(brands).values())[0]

In [124]:
# Embedding brand in emotions space: Get association scores between an input and list of emotions

Characteristic = Enum('Characteristic', dict([(emotion, emotion) for emotion in emotions]))

class EmotionalAssociationScore(BaseModel):
    emotion: Characteristic
    score: float

class EmotionalAssociationScores(BaseModel):
    associations: List[EmotionalAssociationScore] = Field(description="A list of emotions and associated scores")

def emotional_association_scores(
        thing, 
        model,
        emotions
    ):
    
    prompt = f"Assign emotional association scores between {0} and {len(emotions)} for the provided thing. "\
    "Assign a score for each of the following emotions. Briefly, explain the reason behind the association score."\
    "Ensure the scores reflect the association strength for the specified thing. "\
    "Thing: "\
    f"{thing}"
            
    completion = client.beta.chat.completions.parse(
        model = model,
        messages=[
            {"role": "system", "content": "Be a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        response_format=EmotionalAssociationScores,
    )
    #output returns in the defined pydantic style
    output = completion.choices[0].message.parsed
    return output.json()

In [132]:
#Embedding brands in emotions space: 
# tried nested prompt but decided to go with one prompt and a list comprehension
emotions= emotions_ls
associations_brands = [(thing, emotional_association_scores(thing, model, emotions)) for thing in brands_ls[:3]]

In [ ]:
#Embedding a book in emotions space: 
emotions= emotions_ls
book = 'Summer Island'
associations_book = [(thing, emotional_association_scores(thing, model, emotions)) for thing in list(book)]


In [131]:
associations

[('Nike',
  '{"associations": [{"emotion": "Joy", "score": 45.0}, {"emotion": "Shame", "score": 10.0}, {"emotion": "Envy", "score": 25.0}, {"emotion": "Disgust", "score": 5.0}, {"emotion": "Love", "score": 35.0}, {"emotion": "Fear", "score": 5.0}, {"emotion": "Surprise", "score": 20.0}, {"emotion": "Anger", "score": 10.0}, {"emotion": "Sadness", "score": 5.0}, {"emotion": "Hope", "score": 30.0}, {"emotion": "Guilt", "score": 10.0}, {"emotion": "Gratitude", "score": 20.0}, {"emotion": "Contempt", "score": 10.0}, {"emotion": "Pride", "score": 40.0}, {"emotion": "Sympathy", "score": 10.0}, {"emotion": "Awe", "score": 25.0}, {"emotion": "Relief", "score": 15.0}, {"emotion": "Frustration", "score": 15.0}, {"emotion": "Confusion", "score": 10.0}, {"emotion": "Regret", "score": 10.0}, {"emotion": "Amusement", "score": 20.0}, {"emotion": "Empathy", "score": 15.0}, {"emotion": "Despair", "score": 5.0}, {"emotion": "Nostalgia", "score": 20.0}, {"emotion": "Anticipation", "score": 30.0}, {"emotio

In [ ]:
# a method
#embedding dimension is emotions
#talk about options
#get the brands, go through 50 emotins at a time
#cosine: normalize first: l2 norm = 1
#give instructions on readme on where key goes 
#first have everything in pandas df, then think about database
# one module or package w 1 .py 
#adaptors that take in pydantic datatypes and will make into sql
